In [1]:
import agentpy as ap
import networkx as nx
import random
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import IPython

In [144]:
class Person(ap.Agent):
    def setup(self): 
#       Initialise Opinion vector at random.
        self.opinions = np.random.rand(self.p.numberOfOpinionDimensions)*2 - 1
        self.opinions = np.round_(self.opinions, 3)
        
        # Initialise amISpeaking
        self.amISpeaking = False
        
        #initialise boundary
        self.boundary = 0
        
#       Initialise Initial-opinion DO WE NEED THIS?
        self.initialOpinion = self.opinions

#       Set Last-opinion
        self.lastOpinion = self.opinions

#       Initialise "record" from netlogo code
        self.isOpinionUnchanged = np.zeros(self.p.recordLength, int)

#       Initialise P(speaking) and amISpeaking
        self.probabilityOfSpeaking = 0
        self.getPSpeaking()
        self.getIfSpeaking()
        
#       Initialise Pol-bias
#       Initialise HK boundary ie epsilon ie Uncertainty 

        self.getBoundary()

    def getPSpeaking(self):
        if(self.p.howToDrawPSpeaking == "uniform"):
            pass
        elif(self.p.howToDrawPSpeaking == "function"):
            pass
        elif(self.p.howToDrawPSpeaking == "constant"):
            self.probabilityOfSpeaking = self.p.basePSpeaking
            
    def getBoundary(self):
        if(self.p.howToDrawBoundary == "constant"):
            self.boundary = self.p.muBoundary
        elif(self.p.howToDrawPSpeaking == "uniform"):
            pass
        elif(self.p.howToDrawPSpeaking == "normal"):
            pass    
    def getIfSpeaking(self):    
        isISpeak = random.uniform(0, 1) < self.probabilityOfSpeaking
        self.amISpeaking = isISpeak
        
    def changeSpeak(self):
        if(random.uniform(0, 1)< 0.5):
            self.amISpeaking = False
        else:
            self.amISpeaking = True

    def changeOpinionHK(self):
        self.lastOpinion = self.opinions
        self.currentInfluencers = np.array([], int)
        
        #For all agents in the neighbourhood, find those within boundary and add them to array
        for neighbour in self.network.neighbors(self):
            if(np.linalg.norm(self.opinions - neighbour.opinions) < 
               (self.boundary * np.sqrt(self.p.numberOfOpinionDimensions * 4))):
                
                self.currentInfluencers = np.append(self.currentInfluencers, neighbour.id)
            
        self.currentInfluencersOpinions = np.empty((len(self.currentInfluencers), 
                                                   self.p.numberOfOpinionDimensions))
#         print("Agent #" + str(self.id) + " has " + str(len(self.currentInfluencers))
#               + " influencers.")
        for index, influencer in enumerate(self.currentInfluencers):
            
            self.currentInfluencersOpinions[index, :] = self.model.agents[influencer - 1].opinions
                  
        self.opinions = np.mean(self.currentInfluencersOpinions, axis=0)
        self.opinions = np.round_(self.opinions, 3)


In [123]:
class groupFormation(ap.Model):

    def setup(self):
        graph = nx.watts_strogatz_graph(
            self.p.population,
            self.p.numberOfNeighbours,
            self.p.networkRandomness)
        
        self.agents = ap.AgentList(self, self.p.population, Person)
        self.network = self.agents.network = ap.Network(self, graph)
        self.network.add_agents(self.agents, self.network.nodes)


    def step(self):
        self.agents.getIfSpeaking()
        self.agents.changeOpinionHK()

    def update(self):
    
        self.agents.record(['amISpeaking', 'opinions', 'boundary', 'lastOpinion'])

    pass
        

    def end(self):
        """ Repord an evaluation measure. """
#         self.report('my_measure', 1)
        pass

In [145]:
parameters = {
    'population' : 100,
    'numberOfOpinionDimensions' : 4,
    'numberOfNeighbours' : 30,
    'networkRandomness' : 0.5,
    'recordLength' : 10,
    'steps': 2,
    
    'howToDrawPSpeaking' : "constant",
    'basePSpeaking': 0.5,
    
    'howToDrawBoundary' : "constant",
    'muBoundary' : 2,
    'sigmaBoundary' : 0.3
}

model = groupFormation(parameters)
results = model.run()

Completed: 2 steps
Run time: 0:00:00.121069
Simulation finished


In [146]:
results.variables.Person

amISpeaking                         opinions  boundary  \
obj_id t                                                           
1      0        False  [-0.663, 0.439, -0.526, -0.888]         2   
       1         True      [-0.011, 0.07, 0.06, -0.12]         2   
       2        False    [0.027, 0.021, 0.016, -0.013]         2   
2      0         True    [0.857, 0.563, -0.996, 0.301]         2   
       1        False   [-0.075, -0.036, 0.105, -0.13]         2   
...               ...                              ...       ...   
99     1         True    [-0.012, 0.009, 0.006, 0.023]         2   
       2        False    [0.028, 0.012, 0.024, -0.011]         2   
100    0        False   [-0.657, 0.331, -0.124, 0.281]         2   
       1        False     [0.007, 0.021, 0.03, -0.017]         2   
       2        False    [0.029, 0.016, 0.028, -0.013]         2   

                              lastOpinion  
obj_id t                                   
1      0  [-0.663, 0.439, -0.526, -0.888]  
       1  [-0.663, 0.439, -0.526, -0.888]  
       2      [-0.011, 0.07, 0.06, -0.12]  
2      0    [0.857, 0.563, -0.996, 0.301]  
       1    [0.857, 0.563, -0.996, 0.301]  
...                                   ...  
99     1   [-0.096, 0.693, -0.555, 0.862]  
       2    [-0.012, 0.009, 0.006, 0.023]  
100    0   [-0.657, 0.331, -0.124, 0.281]  
       1   [-0.657, 0.331, -0.124, 0.281]  
       2     [0.007, 0.021, 0.03, -0.017]  

[300 rows x 4 columns]

KeyError: Person (Obj 1)

In [91]:
def getIfSpeak():    
       return random.uniform(0, 1) < 0.3

In [116]:
print(str(model.agents[0].currentInfluencers))
np.append(model.agents[0].currentInfluencers, 4)
print(str(model.agents[0].currentInfluencers))

[  2 100  98   7  93  11  91  13  89  88  15  86  24  35  52  67  27  55
  43  53  30  84  50  74  96]
[  2 100  98   7  93  11  91  13  89  88  15  86  24  35  52  67  27  55
  43  53  30  84  50  74  96]


In [135]:
model.agents[1].id

2